In [1]:
import pandas as pd
import os
import csv

In [2]:
# File to Load (Remember to Change These if my path is different)
intakes_data_load = "Data/Austin_Animal_Center_Intakes.csv"
outcomes_data_load = "Data/Austin_Animal_Center_Outcomes.csv"

intakes_data = pd.read_csv(intakes_data_load)
outcomes_data = pd.read_csv(outcomes_data_load)

In [3]:
outcomes_data["Animal ID"].astype(str)
intakes_data["Animal ID"].astype(str)

0         A786884
1         A706918
2         A724273
3         A665644
4         A682524
           ...   
123874    A829565
123875    A825423
123876    A828813
123877    A829553
123878    A829571
Name: Animal ID, Length: 123879, dtype: object

In [4]:
outcomes_data["Animal ID"] = outcomes_data["Animal ID"].map(lambda x: str(x)[1:])
intakes_data["Animal ID"] = intakes_data["Animal ID"].map(lambda x: str(x)[1:])

In [5]:
intakes_data.sort_values(by=['DateTime', "Animal ID"], ascending=False)
outcomes_data.sort_values(by=['DateTime', "Animal ID"], ascending=False)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
5807,827705,NaN,12/31/2020 12:37:00 PM,12/31/2020 12:37:00 PM,07/29/2020,Transfer,Partner,Cat,Intact Female,5 months,Domestic Shorthair,Calico
107089,827706,NaN,12/31/2020 12:36:00 PM,12/31/2020 12:36:00 PM,07/29/2020,Transfer,Partner,Cat,Spayed Female,5 months,Domestic Shorthair,Tortie
35383,827798,Daisy,12/31/2020 12:25:00 PM,12/31/2020 12:25:00 PM,12/31/2018,Return to Owner,NaN,Dog,Intact Female,2 years,Siberian Husky,Black/White
119003,826332,*Gizmo,12/31/2020 12:21:00 PM,12/31/2020 12:21:00 PM,10/09/2020,Adoption,Foster,Dog,Neutered Male,2 months,Dachshund/Dachshund,White/Brown
111265,827571,NaN,12/31/2020 11:46:00 AM,12/31/2020 11:46:00 AM,09/25/2020,Transfer,Partner,Dog,Intact Male,3 months,Chihuahua Shorthair,Black
...,...,...,...,...,...,...,...,...,...,...,...,...
67887,668337,*Tony,01/01/2014 01:42:00 PM,01/01/2014 01:42:00 PM,12/02/2012,Euthanasia,Behavior,Dog,Intact Male,1 year,Pit Bull Mix,Brown Brindle/White
78649,666770,Josh,01/01/2014 01:32:00 PM,01/01/2014 01:32:00 PM,11/07/2010,Adoption,NaN,Dog,Neutered Male,3 years,Siberian Husky Mix,Black/Brown
102473,668314,Duke,01/01/2014 01:30:00 PM,01/01/2014 01:30:00 PM,12/02/2012,Transfer,Partner,Dog,Neutered Male,1 year,Labrador Retriever Mix,Chocolate/White
37444,670044,NaN,01/01/2014 01:13:00 PM,01/01/2014 01:13:00 PM,01/01/2011,Euthanasia,Suffering,Cat,Intact Male,3 years,Domestic Shorthair Mix,Blue


In [6]:
outcomes_data.rename(columns = {'DateTime':'DateTime_out'}, inplace = True)

In [7]:
# Combine the data into a single dataset.  
allanimal_data = pd.merge(intakes_data,outcomes_data[["Animal ID",'DateTime_out', "Outcome Type", "Outcome Subtype"]],on="Animal ID", how='left')
allanimal_data.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,DateTime_out,Outcome Type,Outcome Subtype
0,786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,01/08/2019 03:11:00 PM,Transfer,Partner
1,706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,07/05/2015 03:13:00 PM,Return to Owner,NaN
2,724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,04/21/2016 05:17:00 PM,Return to Owner,NaN
3,665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,10/21/2013 11:39:00 AM,Transfer,Partner
4,682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,07/02/2014 02:16:00 PM,Return to Owner,NaN


In [8]:
allanimal_data["DateTime"]= pd.to_datetime(allanimal_data["DateTime"])

In [9]:
allanimal_data["DateTime_out"]= pd.to_datetime(allanimal_data["DateTime_out"])

In [10]:
allanimal_data["shelter_time"] = allanimal_data["DateTime_out"] - allanimal_data["DateTime"]

In [11]:
adopted_animals = allanimal_data[allanimal_data["Outcome Type"]=="Adoption"]
adopted_animals.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,DateTime_out,Outcome Type,Outcome Subtype,shelter_time
8,818975,NaN,2020-06-18 14:53:00,06/18/2020 02:53:00 PM,Braker Lane And Metric in Travis (TX),Stray,Normal,Cat,Intact Male,4 weeks,Domestic Shorthair,Cream Tabby,2020-07-23 15:54:00,Adoption,Foster,35 days 01:01:00
11,731435,*Casey,2016-08-08 17:52:00,08/08/2016 05:52:00 PM,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,5 months,Domestic Shorthair Mix,Cream Tabby,2016-07-29 19:00:00,Adoption,NaN,-10 days +01:08:00
12,760053,NaN,2017-10-11 15:46:00,10/11/2017 03:46:00 PM,8800 South First Street in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Chihuahua Shorthair,White/Tan,2017-10-15 18:30:00,Adoption,NaN,4 days 02:44:00
13,707375,*Candy Cane,2015-07-11 18:19:00,07/11/2015 06:19:00 PM,Galilee Court And Damita Jo Dr in Manor (TX),Stray,Normal,Dog,Intact Female,5 months,Pit Bull,Brown/White,2015-11-28 17:31:00,Adoption,NaN,139 days 23:12:00
14,696408,*Pearl,2015-02-04 12:58:00,02/04/2015 12:58:00 PM,9705 Thaxton in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Chihuahua Shorthair,Tricolor,2015-05-28 11:24:00,Adoption,Foster,112 days 22:26:00


In [12]:
animal_data = adopted_animals.drop_duplicates(["Animal ID"], keep= 'first')

In [13]:
type_cat = animal_data.loc[animal_data["Animal Type"] == "Cat"]
type_dog = animal_data.loc[animal_data["Animal Type"] == "Dog"]
type_bird = animal_data.loc[animal_data["Animal Type"] == "Bird"]
type_livestock = animal_data.loc[animal_data["Animal Type"] == "Livestock"]
type_other = animal_data.loc[animal_data["Animal Type"] == "Other"]

In [14]:
    type_cats = type_cat[type_cat['shelter_time'] > pd.Timedelta(0)]
    type_dogs = type_dog[type_dog['shelter_time'] > pd.Timedelta(0)]
    type_birds = type_bird[type_bird['shelter_time'] > pd.Timedelta(0)]
    type_livestocks = type_livestock[type_livestock['shelter_time'] > pd.Timedelta(0)]
    type_others = type_other[type_other['shelter_time'] > pd.Timedelta(0)]

In [15]:
type_cats.to_csv("adopted_cats_clean.csv", index=False)
type_dogs.to_csv("adopted_dogs_clean.csv", index=False)
type_birds.to_csv("adopted_birds_clean.csv", index=False)
type_livestocks.to_csv("adopted_livestock_clean.csv", index=False)
type_others.to_csv("adopted_other_clean.csv", index=False)